# 네이버 영화 140자평 데이터 셋 CSV 로 저장하기

In [1]:
#! /usr/bin/python3
# -*- coding: utf-8 -*-

In [2]:
from os import makedirs
import os.path
import csv, codecs

import numpy as np
import pandas as pd
 
import urllib.request as req
import re # 정규식 관련 라이브러리
from bs4 import BeautifulSoup # html 파싱용 라이브러리

In [3]:
BASE_URL = "https://movie.naver.com/movie/point/af/list.nhn?target=after"
PAGE_URL = BASE_URL + "?&page=%s"

## 페이지 수 리턴 함수 정의

In [4]:
def count_page(url):
    html = req.urlopen(url)
    soup = BeautifulSoup(html, "html.parser")
    
    review_count = soup.select("#old_content > h5 > div > strong")
    review_count = int(review_count[0].string)
    
    return int(np.ceil(review_count / 10)) # 한 페이지당 10개의 게시글

## 파싱 함수 정의

In [5]:
def parse_item(item):
    # return 리뷰번호, 평점, 영화제목, 영화리뷰
    # num, point, title, review
    
    _num = item.find('td', {'class':'ac num'})
    _num = _num.text

    _point = item.find('td', {'class':'point'})
    _point = _point.text
    
    movie_info = item.find('td', {'class':'title'})
    movie_info = movie_info.text
    movie_info = movie_info.split('\n')
    
    _title = movie_info[1].strip()
    _review = movie_info[2].strip()
    
    try:
        return {'num': _num, 'point': _point, 'title': _title, 'review': _review}
    except (IndexError, AttributeError) as e:
        print(e, item.text)
    except (AssertionError) as e:
        print(e, "sleep for %s" % 600)
        time.sleep(600)
    except Exception as e:
        print(e)

In [6]:
def scraping_rating_page(url):   
    html = req.urlopen(url)
    soup = BeautifulSoup(html, 'html')
    
    items = soup.findAll('tr')
    items = items[1:] # 첫번째 행은 header 이므로 제거합니다.

    return list([parse_item(item) for item in items])

In [7]:
review_path = '../raw_data'

if not os.path.exists(review_path):
    print("make dirs=", review_path)
    makedirs(review_path)

## CSV 파일로 100 페이지 당 한번씩 Raw 데이터를 저장합니다.

In [8]:
review_df = pd.DataFrame(columns=('num', 'point', 'title', 'review'))
review_file = '../raw_data/%s.csv'

total_page_count = count_page(BASE_URL)
print('총 %s 페이지를 저장합니다.' % total_page_count)

for index in range(1, total_page_count):
    url = PAGE_URL % (index)
    add_list = scraping_rating_page(url)    
    review_df = review_df.append(add_list)
    
    if (index % 100) == 0:
        # 저장
        file_name = review_file % index
        review_df.to_csv(file_name, index=False, header=True)        
        print(file_name, '저장했습니다.')
        
        # 초기화
        review_df = pd.DataFrame(columns=('num', 'point', 'title', 'review'))

총 1127757 페이지를 저장합니다.
../raw_data/100.csv 저장했습니다.
../raw_data/200.csv 저장했습니다.
../raw_data/300.csv 저장했습니다.
../raw_data/400.csv 저장했습니다.
../raw_data/500.csv 저장했습니다.
../raw_data/600.csv 저장했습니다.
../raw_data/700.csv 저장했습니다.
../raw_data/800.csv 저장했습니다.
../raw_data/900.csv 저장했습니다.
../raw_data/1000.csv 저장했습니다.


IndexError: list index out of range

에러가 발생합니다! 왜 일까요?

사실 빈 리스트는 저장하지 않도록 에러처리를 한다면 발생하지 않았을 에러...

네이버 리뷰 사이트로 이동해본다.